In [2]:
from src.environment import NormalMoveEnv
from src.experiment import Experiment
from src.nn import BetaModel, Model, GeneralModel
from src.rl import Agent, Trajectory

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

%matplotlib agg
# cmd -> cd .\notebooks\
# cmd -> mlflow ui

In [3]:
experiment_name = 'N Envs 2'
experiment_description = 'Comparação entre uso de n modelos com o uso de um modelo generalista em um mesmo ambiente que muda ao decorrer da sequencia de episodeos'
num_runs = 10

In [4]:
import mlflow
from mlflow import MlflowException

try:
    exp_id = mlflow.create_experiment(experiment_name, tags={'mlflow.note.content':experiment_description})
    exp = mlflow.get_experiment(exp_id)
except MlflowException as e:
    exp = mlflow.get_experiment_by_name(experiment_name)
    exp_id = exp.experiment_id

print('experiment_id:', exp_id)

experiment_id: 577592340453196052


In [5]:
start = ([0.,-9.9],[9.9,-5.])
goals = [([-9.,8.],[-8.,9.])]
envs_params= [{
        # 'walls': [([-9.,-1.],[10., 1.])],
        'walls': [],
        'goals': goals,
        'start': start,
        'beta': lambda s: (s[1]>-4. and s[1]<4.)
    },
    {
        'walls': [],
        'goals': goals,
        'start': start,
        'beta': lambda s: (s[0]>-5. and s[0]<5.)
    }
]
model_params = {
    "lr":1e-6, 
    'momentum':.9,
    'td_model_steps': 10, 
    'memory_size': 0, 
    'max_plan_size': 10, 
}
k_model_params = {
    'k':5, 
    'model': Model
}
general_params = {
    'model': GeneralModel
}

In [6]:
def gen_env(env_params):
    mlflow.log_params(env_params)
    env = NormalMoveEnv(**env_params)
    mlflow.log_figure(env.plot().get_figure(), f"env.png")
    return env

def run(model, params, episode_size_limit=300):
    model.reset()
    mlflow.log_params(model_params)
    mlflow.log_params(params)
    model.episode(size_limit=episode_size_limit)
    
    mlflow.log_figure(model.plot(kind='probs'), "probs.png")
    mlflow.log_figure(model.plot(), "params.png")
    for plan_id, plan in enumerate(model.plans):
        mlflow.log_figure(model.plot(kind='plan', plan=plan_id), f"plans/plan-{plan_id:03d}.png")

    mlflow.log_metric("Trajectory Size", model.trajectory.run.shape[0])
    mlflow.log_metric("Final Likelyhood", model.lls[-1])

In [7]:
for i in range(num_runs):
    with mlflow.start_run(experiment_id=exp_id, run_name=f'{i}'):
        has_models = False
        for e, env_param in enumerate(envs_params):
            with mlflow.start_run(experiment_id=exp_id, run_name=f'epi-{e}', nested=True):
                env = gen_env(env_param)

                if has_models:
                    k_models.env = env
                    general.env = env
                else:
                    k_models = Agent(env, **k_model_params, **model_params)
                    general = Agent(env, **general_params, **model_params)
                    has_models = True
                    
                with mlflow.start_run(experiment_id=exp_id, run_name='K-Models', nested=True):
                    mlflow.log_param("run", i)
                    mlflow.log_param("episode", e)
                    mlflow.log_figure(env.plot().get_figure(), f"env.png")
                    run(k_models, k_model_params)
                with mlflow.start_run(experiment_id=exp_id, run_name='General', nested=True):
                    mlflow.log_param("run", i)
                    mlflow.log_param("episode", e)
                    mlflow.log_figure(env.plot().get_figure(), f"env.png")
                    run(general, general_params)

C:\Users\guilherme.albarrans\Workspace\kmbrl\src\rl.py:321: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots(figsize=(5, 5))


Parametros estimados para cada estado.
Parametros estimados para cada estado.
Parametros estimados para cada estado.
Parametros estimados para cada estado.
Parametros estimados para cada estado.
Parametros estimados para cada estado.
Parametros estimados para cada estado.
-------Erro no Treinamento-----------
-------Erro no Treinamento-----------
-------Erro no Treinamento-----------
-------Erro no Treinamento-----------
-------Erro no Treinamento-----------
-------Erro no Treinamento-----------
-------Erro no Treinamento-----------
-------Erro no Treinamento-----------
Parametros estimados para cada estado.
Parametros estimados para cada estado.
Parametros estimados para cada estado.
-------Erro no Treinamento-----------
-------Erro no Treinamento-----------
-------Erro no Treinamento-----------
-------Erro no Treinamento-----------
-------Erro no Treinamento-----------
-------Erro no Treinamento-----------
-------Erro no Treinamento-----------
-------Erro no Treinamento-----------
--